# Download last.fm listening history

  - To see this analysis live, check out my article ["Analyzing Last.fm Listening History"](http://geoffboeing.com/2016/05/analyzing-lastfm-history/)
  - My last.fm page: http://www.last.fm/user/gboeing
  - API documentation: http://www.last.fm/api
  - For anything more complicated, you might use this Python wrapper for the API: https://github.com/pylast/pylast
  
This tool separately downloads your all-time most played tracks, artists, and albums. Then it downloads all of your scrobbles in order of recency. Each of these 4 data sets are saved to separate CSV files. It downloads the "all-time most" data separately because (at least for my data) my massive iTunes history scrobble-upload in 2007 is included in the all-time most played tracks/artists/albums but is excluded from the recent tracks API endpoint. For accurate analysis of my all-time scrobbles, I need to look at those separate all-time lists, or else 4 years of listening history (from iTunes) are ignored in the calculations.

In the first cell, replace the "from keys import..." line of code with two new lines of code (replace placeholder values with your actual values):

```python
key=YOUR-LASTFM-API-KEY
username=YOUR-LASTFM-USERNAME
```

In [1]:
import requests, json, time, pandas as pd

key='03dd24d9c70691e2ae5210b0986c2397'
username='jeepster71boy'

/Users/bell/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/bell/anaconda2/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# how long to pause between consecutive API requests
pause_duration = 0.2

## First get your all-time most played tracks, artists, and albums

In [3]:
url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
limit = 200 #api lets you retrieve up to 200 records per call
extended = 0 #api lets you retrieve extended data for each track, 0=no, 1=yes
page = 1 #page of results to start retrieving at

In [4]:
method = 'toptracks'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
track_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['track']:
    artist_names.append(item['artist']['name'])
    track_names.append(item['name'])
    play_counts.append(item['playcount'])

top_tracks = pd.DataFrame()
top_tracks['artist'] = artist_names
top_tracks['track'] = track_names
top_tracks['play_count'] = play_counts
top_tracks.to_csv('data/lastfm_top_tracks.csv', index=None, encoding='utf-8')
top_tracks.head()

,artist,track,play_count
0,Miranda Lambert,For the Birds,47
1,Miranda Lambert,We Should Be Friends,43
2,Red Hot Chili Peppers,Soul to Squeeze,39
3,Miranda Lambert,Well-Rested,38
4,Miranda Lambert,To Learn Her,36


In [5]:
method = 'topartists'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['artist']:
    artist_names.append(item['name'])
    play_counts.append(item['playcount'])

top_artists = pd.DataFrame()
top_artists['artist'] = artist_names
top_artists['play_count'] = play_counts
top_artists.to_csv('data/lastfm_top_artists.csv', index=None, encoding='utf-8')
top_artists.head()

,artist,play_count
0,Miranda Lambert,1235
1,Explosions in the Sky,670
2,Dean Martin,534
3,Norah Jones,434
4,Jack Johnson,353


In [6]:
method = 'topalbums'
request_url = url.format(method, username, key, limit, extended, page)
artist_names = []
album_names = []
play_counts = []
response = requests.get(request_url).json()
for item in response[method]['album']:
    artist_names.append(item['artist']['name'])
    album_names.append(item['name'])
    play_counts.append(item['playcount'])

top_albums = pd.DataFrame()
top_albums['artist'] = artist_names
top_albums['album'] = album_names
top_albums['play_count'] = play_counts
top_albums.to_csv('data/lastfm_top_albums.csv', index=None, encoding='utf-8')
top_albums.head()

,artist,album,play_count
0,Miranda Lambert,The Weight of These Wings,693
1,Miranda Lambert,Platinum,289
2,Dean Martin,Dino: The Essential Dean Martin,276
3,Explosions in the Sky,The Wilderness,140
4,Explosions in the Sky,Manglehorn (Original Motion Picture Soundtrack),138


## Now get all your scrobbles

Last.fm provides this 'recenttracks' API method to get 'all' scrobbles. However, it seems to be pretty spotty for data from circa 2007. The best way to determine top tracks, artists, albums is with the cells above. However, the code below retrieves time series data of all scrobbles (but with the caveat of spotty data from 2007 and earlier).

Sample URL: https://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user=gboeing&api_key={}&limit=1&extended=0&page=1&format=json

In [7]:
def get_scrobbles(method='recenttracks', username=username, key=key, limit=200, extended=0, page=1, pages=0):
    '''
    method: api method
    username/key: api credentials
    limit: api lets you retrieve up to 200 records per call
    extended: api lets you retrieve extended data for each track, 0=no, 1=yes
    page: page of results to start retrieving at
    pages: how many pages of results to retrieve. if 0, get as many as api can return.
    '''
    # initialize url and lists to contain response fields
    url = 'https://ws.audioscrobbler.com/2.0/?method=user.get{}&user={}&api_key={}&limit={}&extended={}&page={}&format=json'
    responses = []
    artist_names = []
    artist_mbids = []
    album_names = []
    album_mbids = []
    track_names = []
    track_mbids = []
    timestamps = []
    
    # make first request, just to get the total number of pages
    request_url = url.format(method, username, key, limit, extended, page)
    response = requests.get(request_url).json()
    total_pages = int(response[method]['@attr']['totalPages'])
    if pages > 0:
        total_pages = min([total_pages, pages])
        
    print('{} total pages to retrieve'.format(total_pages))
    
    # request each page of data one at a time
    for page in range(1, int(total_pages) + 1, 1):
        if page % 10 == 0: print(page, end=' ')
        time.sleep(pause_duration)
        request_url = url.format(method, username, key, limit, extended, page)
        responses.append(requests.get(request_url))
    
    # parse the fields out of each scrobble in each page (aka response) of scrobbles
    for response in responses:
        scrobbles = response.json()
        for scrobble in scrobbles[method]['track']:
            # only retain completed scrobbles (aka, with timestamp and not 'now playing')
            if 'date' in scrobble.keys():
                artist_names.append(scrobble['artist']['#text'])
                artist_mbids.append(scrobble['artist']['mbid'])
                album_names.append(scrobble['album']['#text'])
                album_mbids.append(scrobble['album']['mbid'])
                track_names.append(scrobble['name'])
                track_mbids.append(scrobble['mbid'])
                timestamps.append(scrobble['date']['uts'])
                
    # create and populate a dataframe to contain the data
    df = pd.DataFrame()
    df['artist'] = artist_names
    df['artist_mbid'] = artist_mbids
    df['album'] = album_names
    df['album_mbid'] = album_mbids
    df['track'] = track_names
    df['track_mbid'] = track_mbids
    df['timestamp'] = timestamps
    df['datetime'] = pd.to_datetime(df['timestamp'].astype(int), unit='s')
    
    return df

In [8]:
# get all scrobbled tracks ever, in order of recency (pages=0 to get all)
scrobbles = get_scrobbles(pages=0)

102 total pages to retrieve
10 20 30 40 50 60 70 80 90 100 

In [9]:
# save the dataset
scrobbles.to_csv('data/lastfm_scrobbles.csv', index=None, encoding='utf-8')
print('{:,} total rows'.format(len(scrobbles)))
scrobbles.head()

20,251 total rows


,artist,artist_mbid,album,album_mbid,track,track_mbid,timestamp,datetime
0,Laura Fygi,f51abb6f-fb5e-47c8-9089-c9e7b7fe7a55,Jazz Love,,Vincent,,1534276564,2018-08-14 19:56:04
1,Madeleine Peyroux,7f260b9c-bea7-469b-9ba8-0daec7d2c85c,Secular Hymns,,Got You On My Mind,8fc15006-14f8-4ac9-812e-7eb98ea8dd52,1534276294,2018-08-14 19:51:34
2,Ella Fitzgerald,54799c0e-eb45-4eea-996d-c4d71a63c499,100 Songs For A Centennial,,Taking a Chance On Love,8e9e2a34-484a-4671-bfd6-a00b96716ba8,1534276106,2018-08-14 19:48:26
3,Duke Ellington,eb34b261-d9da-4ab4-abd3-34e7690d80ef,The Duke (Remastered 2014),,Lonesome Lullaby,,1534275908,2018-08-14 19:45:08
4,Ella Fitzgerald,54799c0e-eb45-4eea-996d-c4d71a63c499,Legendary Ladies of Jazz,,A-tisket A-tasket,b09c1c88-a51f-4906-8288-8437fa3518f9,1534275752,2018-08-14 19:42:32
